##### Imports

In [1]:
import sys
from pathlib import Path
import warnings

import warnings
import pandas as pd
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 1000)

import sys
# Custom library paths
sys.path.extend(['../', './scr'])

from scr.utils import set_seed
from scr.utils import read_words
from pathlib import Path
import random
from collections import Counter, defaultdict
import pickle
from tqdm import tqdm
from torch.utils.data import Dataset

from scr.utils import read_words, save_words_to_file

import pickle
from pathlib import Path
from scr.dataset import *
from scr.utils import *
# # For inference
from scr.feature_engineering import *

import gc

set_seed(42)

import torch
import torch.nn as nn
from pathlib import Path
import random

from scr.utils import print_scenarios
torch.set_float32_matmul_precision('medium')
from pathlib import Path

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# # Read and Shuffle Word List
word_list = read_words('data/words_250000_train.txt') # , limit=10000)
# word_list = read_words('data/250k.txt', limit=10000)

random.shuffle(word_list)

# Calculate Frequencies and Max Word Length
word_frequencies = calculate_word_frequencies(word_list)
char_frequency = calculate_char_frequencies(word_list)
max_word_length = max(len(word) for word in word_list)

##### Data Dir

In [2]:
NUM_STRATIFIED_SAMPLES = 100 # This will be overwritten by Papermill

NUM_WORD_SAMPLE = 1000 # words for testing

FAST_DEV_RUN = False

MAX_EPOCH = 15

In [3]:
from pathlib import Path

# Define the base directory and the paths for training and validation parquet files
base_dataset_dir = Path("/media/sayem/510B93E12554BBD1/dataset/")
stratified_samples_dir = base_dataset_dir / str(NUM_STRATIFIED_SAMPLES)
parquet_train_path = stratified_samples_dir / 'train_parquets'
parquet_valid_path = stratified_samples_dir / 'valid_parquets'

# Create directories for train and validation parquets if they don't exist
parquet_train_path.mkdir(parents=True, exist_ok=True)
parquet_valid_path.mkdir(parents=True, exist_ok=True)

# Define and create the directory for models
models_dir = Path("/home/sayem/Desktop/Hangman/models")
models_dir.mkdir(parents=True, exist_ok=True)

# Define your output directory
# Define your output directory and logger directory
output_dir = Path("/home/sayem/Desktop/Hangman/training_outputs")
logger_dir = output_dir / "lightning_logs"

# Create the output and logger directories if they don't exist
output_dir.mkdir(parents=True, exist_ok=True)
logger_dir.mkdir(parents=True, exist_ok=True)

# Define the file path for saving the testing words
testing_words_file_path = stratified_samples_dir / "testing_words.txt"

try:
    testing_word_list = read_words(testing_words_file_path)
    sampled_test_words = sample_words(testing_word_list, NUM_WORD_SAMPLE)
    print(f"Sampled {len(sampled_test_words)} unique words for testing.")
except FileNotFoundError:
    print(f"File not found: {testing_words_file_path}")

print(len(sampled_test_words))

Sampled 1000 unique words for testing.
1000


##### Dataset Loading

In [4]:
# Create datasets directly from the saved parquet files
train_dataset = HangmanDataset(parquet_train_path)
valid_dataset = HangmanDataset(parquet_valid_path)

print(len(train_dataset))
print(len(valid_dataset))

assert len(train_dataset) > len(valid_dataset)

6894
1764


##### Model Initialization

In [5]:
# from scr.model import HangmanNN
# # Usage
# from scr.model import *
# from scr.trainer import *
# # from scr.trainer import *  # Import other necessary modules if required

# # Define your parameters
# embedding_dim = 145
# hidden_dim = 256
# output_dim = 28
# num_layers = 2
# missed_char_dim = 28
# num_embeddings = 100  # Example value, adjust as needed
# num_additional_features = 4  # Example value, adjust as needed
# dropout_rate = 0.5  # Optional, since it has a default value

# # Initialize the HangmanNN model
# lstm_model = HangmanNN(embedding_dim=embedding_dim, hidden_dim=hidden_dim,
#                           output_dim=output_dim, num_layers=num_layers,
#                           missed_char_dim=missed_char_dim, 
#                           num_embeddings=num_embeddings,
#                           num_additional_features=num_additional_features, 
#                           dropout_rate=dropout_rate)

In [6]:
# Usage
from scr.model import *
from scr.trainer import *

lstm_model = SimpleLSTM(input_dim=145, hidden_dim=256, output_dim=28, 
                                num_layers=2, missed_char_dim=28)

# Save the model's state dictionary
model_file = models_dir / f"{NUM_STRATIFIED_SAMPLES}_untrained_model.pth"
torch.save(lstm_model, model_file)
print(f"Model saved at {model_file}")

Model saved at /home/sayem/Desktop/Hangman/models/100_untrained_model.pth


##### Testing on Untrained Model

In [7]:
from pytorch_lightning.profilers import SimpleProfiler
import pytorch_lightning as pl
from scr.custom_callbacks import *
from scr.dataset import *
from scr.data_module import *

learning_rate = 0.19

lightning_model = HangmanModel(lstm_model, learning_rate, char_frequency, 
                               max_word_length, test_words=sampled_test_words)

# Initialize Data Module
initial_batch_size = 32  # Set your initial batch size

# Initialize Data Module with the required arguments
data_module = HangmanDataModule(train_dataset, valid_dataset, 
                                initial_batch_size, 
                                custom_collate_fn)

# # Create Callbacks
# loss_logging_callback = LossLoggingCallback()

callbacks = [early_stop_callback] # , loss_logging_callback]

# FAST_DEV_RUN = False
# # Calculate the minimum percentage of validation batches
# min_val_batches = 1 / len(data_module.val_dataloader())

# # Create Trainer with Callbacks
trainer = pl.Trainer(
    default_root_dir=output_dir,
    fast_dev_run=FAST_DEV_RUN, 
    max_epochs=MAX_EPOCH, 
    callbacks=callbacks,
    # limit_train_batches=2,  # Limit the number of training batches to 2
    # limit_val_batches=2     # Limit the number of validation batches to 2
)

# # # # Validate the model (if needed)
trainer.validate(model=lightning_model, datamodule=data_module)
# # print(f"Training Begin...")
# # # Fit the model
# trainer.fit(lightning_model, data_module)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Validation: |          | 0/? [00:00<?, ?it/s]

Processing words: 100%|██████████| 1000/1000 [00:04<00:00, 229.52word/s]


────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
     Validate metric           DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
  overall_avg_attempts       5.979000091552734
      test_win_rate         0.6000000238418579
     val_loss_epoch         0.45484036207199097
 val_miss_penalty_epoch    0.054456859827041626
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'val_loss_epoch': 0.45484036207199097,
  'val_miss_penalty_epoch': 0.054456859827041626,
  'test_win_rate': 0.6000000238418579,
  'overall_avg_attempts': 5.979000091552734}]

##### Tuning: lr

In [8]:
from pytorch_lightning.tuner.tuning import Tuner

# Assuming lightning_model, train_loader, and val_loader are already defined
# Initialize the tuner with your trainer
tuner = Tuner(trainer)

# Run the learning rate finder using the data module
lr_finder = tuner.lr_find(model=lightning_model, datamodule=data_module)

# Plot the learning rate finder results
fig = lr_finder.plot(suggest=True)
fig.show()

# Get the suggested learning rate
new_lr = lr_finder.suggestion()
print(f"Suggested Learning Rate: {new_lr}")

# Update model's learning rate
lightning_model.learning_rate = new_lr

# Optionally, you can view the results of the LR finder
print(lr_finder.results)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


##### Tuning: Batch

In [ ]:
# Assuming lightning_model is already defined
new_batch_size = tuner.scale_batch_size(
    model=lightning_model,
    datamodule=data_module,
    mode='power',  # or 'binsearch'
    steps_per_trial=3,
    init_val=256,
    max_trials=4
)

# Update the batch size in the data module
data_module.batch_size = new_batch_size

print(f"Tune Batch size: ", new_batch_size)

##### Training

In [ ]:
print(f"Training Begin...")

# Fit the model
trainer.fit(lightning_model, data_module)

# Optionally print the profiler summary
# print(profiler.summary())

# Save the entire model
trained_model_file = models_dir / f"{NUM_STRATIFIED_SAMPLES}_trained_model.pth"
torch.save(lightning_model, trained_model_file)
print(f"Model saved at {trained_model_file}")

##### Testing

In [ ]:
trained_model_file_path = models_dir / f"{NUM_STRATIFIED_SAMPLES}_trained_model.pth"
trained_model = torch.load(trained_model_file_path)
print(type(trained_model))

<class 'scr.trainer.HangmanModel'>


In [ ]:
# Load the entire LSTM model object
trained_model_file_path = models_dir / f"{NUM_STRATIFIED_SAMPLES}_trained_model.pth"
trained_model = torch.load(trained_model_file_path)

# # If you want to use the model for inference
# trained_model.eval()  # Set the model to evaluation mode

from scr.game import *

word = 'may'

play_game_with_a_word(trained_model, \
    word, char_frequency, max_word_length)

(False, '_a_', 6)

In [ ]:
NUM_STRATIFIED_SAMPLES

100

In [ ]:
# Example usage
result = play_games_and_calculate_stats(trained_model, \
    sampled_test_words, char_frequency, max_word_length)

print(f"Overall Win Rate: {result['overall_win_rate']}%, Overall Average Attempts: {result['overall_avg_attempts']}")

for length, data in result["length_wise_stats"].items():
    print(f"Length {length}: Win Rate: {data['win_rate']}%, Average Attempts: {data['average_attempts_used']}")

Processing words: 100%|██████████| 1000/1000 [00:06<00:00, 165.33word/s]

Overall Win Rate: 15.8%, Overall Average Attempts: 5.686
Length 5: Win Rate: 9.475806451612904%, Average Attempts: 5.82258064516129
Length 17: Win Rate: 33.33333333333333%, Average Attempts: 5.246913580246914
Length 13: Win Rate: 19.858156028368796%, Average Attempts: 5.609929078014185
